In [123]:
import pandas as pd
import numpy as np

In [126]:
def global_depth(rank):
    """
    will return the first power of 8 greater than rank, the power corresponding to the level
    """
    level = 1
    loop = True

    while loop:

        if rank <= 8**level:
            loop = False
            result = level
        else:
            level += 1
    
    return result

v_global_depth = np.vectorize(global_depth)


def local_depth(points_per_rank, max_points_per_leaf, min_local_depth):

    local_level = min_local_depth
    loop = True

    while loop:
        n_leaves = 8**local_level # leaves in local tree
        points_per_leaf = points_per_rank / n_leaves
        if points_per_leaf <= max_points_per_leaf:
            loop = False
            result = local_level
        else:
            local_level += 1
        
    return result

v_local_depth = np.vectorize(local_depth)

def pow2(x): return np.log2(x).is_integer()

In [136]:
max_nodes = 256
min_nodes = 1
ranks_per_node=32 # ranks on 1 node, such that 1 per CCX region
total_ranks = max_nodes*ranks_per_node # 16 nodes, 32 ranks per node (1 per CCX region)
min_points_per_rank = 125e3
n = min_points_per_rank*total_ranks # total problem size
print(f"Total problem size n={n/1e9}B")

# Double number of nodes used until we reach max_nodes
n_nodes = np.array(list(filter(pow2, [i for i in range(min_nodes, max_nodes+1)])))

points_per_node = [n]
for i in range(len(n_nodes)-1):
    p = points_per_node[i]/2
    points_per_node.append(p)
points_per_node = np.array(points_per_node)
    
total_ranks = n_nodes*32
points_per_rank = np.int64(points_per_node/ranks_per_node)


# The global depth is a function of the ranks per node (as will need enough global leaves to cover this many ranks (local trees))
global_depth = v_global_depth(total_ranks)
print(f"global depth {global_depth}")

# Calculate max threshold for points per leaf from largest scale problem
min_local_depth = 3
max_points_per_leaf = points_per_rank[-1] / 8**min_local_depth


local_depth = v_local_depth(points_per_rank, max_points_per_leaf = 2000, min_local_depth=min_local_depth)
print(f"local depth {local_depth}")

print(f"points per rank {points_per_rank}")

Total problem size n=1.024B
global depth [2 2 3 3 3 4 4 4 5]
local depth [5 5 4 4 4 3 3 3 3]
points per rank [32000000 16000000  8000000  4000000  2000000  1000000   500000   250000
   125000]


In [137]:
total_ranks * points_per_rank

array([1024000000, 1024000000, 1024000000, 1024000000, 1024000000,
       1024000000, 1024000000, 1024000000, 1024000000])

In [139]:
total_ranks

array([  32,   64,  128,  256,  512, 1024, 2048, 4096, 8192])

In [140]:
n_nodes

array([  1,   2,   4,   8,  16,  32,  64, 128, 256])